In [2]:
# ============================================================
# CELL 1: Install, Copy Code, and Merge HAM10000
# ============================================================
!pip install -q albumentations timm torchmetrics

import os
import sys

# Set Kaggle Working Directory
KAGGLE_WORK = "/kaggle/working"
os.chdir(KAGGLE_WORK)
sys.path.insert(0, KAGGLE_WORK)

# EXACT PATHS BASED ON YOUR DIAGNOSTICS
SETUP_DIR = "/kaggle/input/datasets/naeemurrahman32/skin-traige/ml_core"
HAM_DIR = "/kaggle/input/datasets/kmader/skin-cancer-mnist-ham10000"

# Copy code, configs, and CSVs
!cp -r {SETUP_DIR}/src ./src
!cp -r {SETUP_DIR}/config ./config
!cp -r {SETUP_DIR}/data/processed ./data_processed

# Copy checkpoint
!mkdir -p checkpoints
!cp {SETUP_DIR}/checkpoints/efficientnet_b0_latest.pth ./checkpoints/

# Merge HAM10000 parts
print("Merging HAM10000 parts into a single fast folder...")
!mkdir -p /kaggle/working/ham10000_images
!cp -r {HAM_DIR}/HAM10000_images_part_1/* /kaggle/working/ham10000_images/
!cp -r {HAM_DIR}/HAM10000_images_part_2/* /kaggle/working/ham10000_images/

print("✅ Environment setup and data merging complete!")

Merging HAM10000 parts into a single fast folder...
✅ Environment setup and data merging complete!


In [3]:
# ============================================================
# CELL 2: Kaggle Paths & Dataloaders
# ============================================================
import torch
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

from src.utils.config import load_config
from src.datasets.dataloader import create_dataloaders_from_config, compute_class_weights

config = load_config("config/config.yaml")

# 🚨 OVERRIDE PATHS TO EXACT KAGGLE INPUTS 🚨
config.paths.ham_images = "/kaggle/working/ham10000_images" 
config.paths.isic_images = "/kaggle/input/datasets/nischaydnk/isic-2019-jpg-224x224-resized/train-image/image" 

# Route CSVs and checkpoints to the working directory
config.paths.train_csv = "./data_processed/train.csv"
config.paths.val_csv = "./data_processed/val.csv"
config.paths.test_csv = "./data_processed/test.csv"
config.paths.checkpoints_dir = "./checkpoints"

# T4x2 GPUs love 4 workers!
config.training.num_workers = 4  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("\n" + "="*40)
print(f"🔥 HARDWARE CHECK: {device.type.upper()} 🔥")
print("="*40 + "\n")

dataloaders, datasets = create_dataloaders_from_config(config)
train_loader = dataloaders['train']
val_loader   = dataloaders['val']

print(f"✓ Train batches: {len(train_loader)}")

class_weights = compute_class_weights(config.paths.train_csv, config.dataset.class_to_idx).to(device)
print("\n✅ Fast Kaggle Dataloaders Ready!")

INFO: NumExpr defaulting to 4 threads.
INFO: Loading configuration from config/config.yaml



🔥 HARDWARE CHECK: CUDA 🔥



INFO: TransformFactory initialized (size=224)
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
INFO: Loading dataset from /kaggle/working/data_processed/train.csv
INFO: ✓ Dataset initialized: 17,731 images
INFO:   Sources: {'isic2019': 10733, 'ham10000': 6998}
INFO:   Classes: {'nv': 9012, 'mel': 3165, 'bcc': 2326, 'bkl': 1837, 'akiec': 745, 'scc': 302, 'vasc': 177, 'df': 167}
INFO: Loading dataset from /kaggle/working/data_processed/val.csv
INFO: ✓ Dataset initialized: 3,800 images
INFO:   Sources: {'isic2019': 2288, 'ham10000': 1512}
INFO:   Classes: {'nv': 1931, 'mel': 678, 'bcc': 499, 'bkl': 393, 'akiec': 160, 'scc': 65, 'vasc': 38, 'df': 36}
INFO: Loading dataset from /kaggle/working/data_processed/test.csv
INFO: ✓ Dataset initialized: 3,800 images
INFO:   Sources: {'isic2019': 2295, 'ham10000': 1505}
INFO

✓ Train batches: 555

✅ Fast Kaggle Dataloaders Ready!


In [6]:
%%writefile src/training/trainer.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
import numpy as np
import time
import logging
from pathlib import Path
from typing import Dict, Tuple
from tqdm import tqdm

from .metrics import MetricsCalculator

logger = logging.getLogger(__name__)

# ==========================================
# NEW: THE FOCAL LOSS UPGRADE
# ==========================================
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

class EarlyStopping:
    def __init__(self, patience: int = 7, min_delta: float = 0.001, mode: str = 'max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_value = None
        self.should_stop = False

    def __call__(self, value: float) -> bool:
        if self.best_value is None:
            self.best_value = value
            return False
        improved = (value > self.best_value + self.min_delta if self.mode == 'max' else value < self.best_value - self.min_delta)
        if improved:
            self.best_value = value
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        return self.should_stop

class Trainer:
    def __init__(self, model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, class_weights: torch.Tensor, config, device: torch.device, model_name: str, checkpoint_dir: str = "checkpoints"):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.model_name = model_name
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)
        self.num_epochs = config.training.num_epochs
        self.use_amp = config.training.use_amp and device.type == 'cuda'
        self.grad_clip = config.training.grad_clip_norm

        # 🚨 APPLIED FOCAL LOSS HERE 🚨
        self.criterion = FocalLoss(weight=class_weights.to(device), gamma=2.0)

        self.optimizer = optim.AdamW(model.parameters(), lr=config.training.learning_rate, weight_decay=config.training.weight_decay)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='max', factor=config.training.scheduler_factor, patience=config.training.scheduler_patience, min_lr=config.training.min_lr)
        self.scaler = GradScaler(enabled=self.use_amp)
        self.early_stopping = EarlyStopping(patience=config.training.early_stopping_patience, mode='max')
        self.metrics_calc = MetricsCalculator(results_dir=str(Path(config.paths.results_dir)))
        self.history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_balanced_acc': [], 'val_mel_sensitivity': [], 'lr': []}
        self.best_val_balanced_acc = 0.0

    def _train_one_epoch(self, epoch: int) -> Tuple[float, float]:
        self.model.train()
        total_loss, correct, total = 0.0, 0, 0
        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False, dynamic_ncols=True)
        for images, labels in pbar:
            images, labels = images.to(self.device, non_blocking=True), labels.to(self.device, non_blocking=True)
            self.optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=self.use_amp):
                logits = self.model(images)
                loss = self.criterion(logits, labels)
            self.scaler.scale(loss).backward()
            self.scaler.unscale_(self.optimizer)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
            self.scaler.step(self.optimizer)
            self.scaler.update()
            total_loss += loss.item()
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            pbar.set_postfix({'loss': f"{loss.item():.4f}", 'acc': f"{100 * correct / total:.1f}%"})
        return total_loss / len(self.train_loader), correct / total

    @torch.no_grad()
    def _validate(self, epoch: int) -> Tuple[float, Dict]:
        self.model.eval()
        total_loss = 0.0
        all_preds, all_labels, all_probs = [], [], []
        pbar = tqdm(self.val_loader, desc=f"Epoch {epoch+1} [Val]", leave=False, dynamic_ncols=True)
        for images, labels in pbar:
            images, labels = images.to(self.device, non_blocking=True), labels.to(self.device, non_blocking=True)
            with autocast(enabled=self.use_amp):
                logits = self.model(images)
                loss = self.criterion(logits, labels)
            total_loss += loss.item()
            probs = torch.softmax(logits, dim=1)
            preds = probs.argmax(dim=1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        all_preds, all_labels, all_probs = np.concatenate(all_preds), np.concatenate(all_labels), np.concatenate(all_probs)
        return total_loss / len(self.val_loader), self.metrics_calc.compute(all_preds, all_labels, all_probs, split='val'), all_preds, all_labels, all_probs

    def _save_checkpoint(self, epoch: int, metrics: Dict, is_best: bool = False):
        checkpoint = {'epoch': epoch, 'model_name': self.model_name, 'model_state_dict': self.model.state_dict(), 'optimizer_state_dict': self.optimizer.state_dict(), 'val_balanced_acc': metrics['balanced_accuracy'], 'val_mel_sensitivity': metrics['mel_sensitivity'], 'metrics': metrics}
        torch.save(checkpoint, self.checkpoint_dir / f"{self.model_name}_latest.pth")
        if is_best:
            best_path = self.checkpoint_dir / f"{self.model_name}_best.pth"
            torch.save(checkpoint, best_path)
            logger.info(f"  ✅ New best model saved: {best_path.name}")

    def _plot_training_curves(self):
        import matplotlib.pyplot as plt
        fig, axes = plt.subplots(1, 3, figsize=(16, 4))
        axes[0].plot(self.history['train_loss'], label='Train')
        axes[0].plot(self.history['val_loss'], label='Val')
        axes[0].set_title(f'{self.model_name} — Loss')
        axes[0].legend()
        axes[1].plot(self.history['train_acc'], label='Train Acc')
        axes[1].plot(self.history['val_balanced_acc'], label='Val Balanced Acc')
        axes[1].set_title(f'{self.model_name} — Accuracy')
        axes[1].legend()
        axes[2].plot(self.history['val_mel_sensitivity'], color='red')
        axes[2].axhline(y=0.90, color='darkred', linestyle='--')
        axes[2].set_title(f'{self.model_name} — Mel Sensitivity')
        axes[2].legend()
        axes[2].set_ylim([0, 1])
        plt.tight_layout()
        plt.savefig(Path("results") / f"training_curves_{self.model_name}.png", dpi=150, bbox_inches='tight')
        plt.close()

    def train(self) -> Dict:
        logger.info(f"\n🚀 Training {self.model_name}...")
        start_time = time.time()
        for epoch in range(self.num_epochs):
            epoch_start = time.time()
            train_loss, train_acc = self._train_one_epoch(epoch)
            val_loss, metrics, _, _, _ = self._validate(epoch)
            current_lr = self.optimizer.param_groups[0]['lr']
            self.scheduler.step(metrics['balanced_accuracy'])
            self.history['train_loss'].append(train_loss); self.history['val_loss'].append(val_loss)
            self.history['train_acc'].append(train_acc); self.history['val_balanced_acc'].append(metrics['balanced_accuracy'])
            self.history['val_mel_sensitivity'].append(metrics['mel_sensitivity']); self.history['lr'].append(current_lr)
            is_best = metrics['balanced_accuracy'] > self.best_val_balanced_acc
            if is_best: self.best_val_balanced_acc = metrics['balanced_accuracy']
            self._save_checkpoint(epoch, metrics, is_best)
            print(f"Ep {epoch+1:02d}/{self.num_epochs} | Loss: {train_loss:.4f}/{val_loss:.4f} | BalAcc: {metrics['balanced_accuracy']:.4f} | MelSens: {metrics['mel_sensitivity']:.4f} | F1: {metrics['f1_macro']:.4f} | LR: {current_lr:.6f} | ⏱ {time.time() - epoch_start:.0f}s")
            if self.early_stopping(metrics['balanced_accuracy']):
                logger.info(f"\n⏹ Early stopping at epoch {epoch+1}")
                break
        logger.info(f"\n✅ Training complete: {(time.time() - start_time) / 60:.1f} min")
        self._plot_training_curves()
        return self.history

Overwriting src/training/trainer.py


In [5]:
# --- FORCE PYTHON TO LOAD THE NEW FILE ---
import importlib
import src.training.trainer
importlib.reload(src.training.trainer)
from src.training.trainer import Trainer
print("✅ Trainer reloaded with Focal Loss!")

✅ Trainer reloaded with Focal Loss!


In [5]:
# ============================================================
# CELL 3: RESUME Train Model 1 — EfficientNet-B0 (KAGGLE)
# ============================================================
import torch
from pathlib import Path

# Import your custom modules from the working directory
from src.models.classifier import build_model
from src.training.trainer import Trainer

MODEL_NAME = "efficientnet_b0"

# 1. Build the model architecture
model = build_model(
    backbone_name=MODEL_NAME,
    num_classes=config.dataset.num_classes,
    dropout=0.3,
    pretrained=False, 
    device=device
)

# 2. Setup the Trainer
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    class_weights=class_weights,
    config=config,
    device=device,
    model_name=MODEL_NAME,
    checkpoint_dir=config.paths.checkpoints_dir,
)

# 3. Look for the saved checkpoint in the Kaggle working directory
checkpoint_path = Path(config.paths.checkpoints_dir) / f"{MODEL_NAME}_latest.pth"

if checkpoint_path.exists():
    print(f"✅ Found checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    trainer.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    trainer.best_val_balanced_acc = checkpoint.get('val_balanced_acc', 0.0)
    
    epochs_completed = checkpoint.get('epoch', 0) + 1
    print(f"🚀 Successfully loaded weights! Resuming after {epochs_completed} epochs...")
    
    print(f"\nPhase 2: Resuming full fine-tuning on {device}...")
    model.unfreeze_backbone()
    
    epochs_total = config.training.num_epochs
    epochs_remaining = epochs_total - epochs_completed
    
    if epochs_remaining > 0:
        trainer.num_epochs = epochs_remaining
        trainer.early_stopping.counter = 0  
        history_efficientnet = trainer.train()
        print("\n✅ Resumed training complete!")
        print(f"   Best Balanced Acc: {trainer.best_val_balanced_acc:.4f}")
    else:
        print("\n✅ Model has already completed all 30 epochs!")
else:
    print(f"❌ ERROR: No checkpoint found at {checkpoint_path}!")
    print("Check your Cell 1 outputs to ensure the .pth file copied correctly.")

INFO: Model: efficientnet_b0
INFO:   Feature dim: 1280
INFO:   Params: 4,017,796
INFO:   Pretrained: False
/kaggle/working/src/training/trainer.py:121: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler(enabled=self.use_amp)
INFO: 
INFO: Trainer initialized: efficientnet_b0
INFO:   Device: cuda | AMP: True
INFO:   Epochs: 30 | Batch: 32
INFO:   LR: 0.0001 | WD: 0.0001
INFO: ============================================================
INFO: Backbone unfrozen (full fine-tuning)
INFO: 
🚀 Training efficientnet_b0...


✅ Found checkpoint: checkpoints/efficientnet_b0_latest.pth
🚀 Successfully loaded weights! Resuming after 5 epochs...

Phase 2: Resuming full fine-tuning on cuda...


Epoch 1 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:171: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 1 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:220: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.8061
INFO:   Balanced Accuracy:  0.8084
INFO:   F1 Macro:           0.7499
INFO:   AUC-ROC Macro:      0.0000
INFO:   🎯 Mel Sensitivity: 0.7375
INFO:   🎯 Mel AUC-ROC:     0.0000
INFO:   ✅ New best model saved: efficientnet_b0_best.pth
INFO:      Balanced Acc: 0.8084 | Mel Sensitivity: 0.7375


Ep 01/25 | Loss: 0.3137/0.7826 | BalAcc: 0.8084 | MelSens: 0.7375 | F1: 0.7499 | LR: 0.000100 | ⏱ 104s


Epoch 2 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:171: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 2 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:220: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.8192
INFO:   Balanced Accuracy:  0.7982
INFO:   F1 Macro:           0.7812
INFO:   AUC-ROC Macro:      0.0000
INFO:   🎯 Mel Sensitivity: 0.7080
INFO:   🎯 Mel AUC-ROC:     0.0000


...

Ep 24/25 | Loss: 0.0846/1.2425 | BalAcc: 0.8084 | MelSens: 0.6947 | F1: 0.8177 | LR: 0.000100 | ⏱ 70s


Epoch 25 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:171: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 25 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:220: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.8537
INFO:   Balanced Accuracy:  0.8138
INFO:   F1 Macro:           0.8082
INFO:   AUC-ROC Macro:      0.0000
INFO:   🎯 Mel Sensitivity: 0.7566
INFO:   🎯 Mel AUC-ROC:     0.0000
INFO: 
✅ Training complete: 29.4 min
INFO:    Best Balanced Acc: 0.8265


Ep 25/25 | Loss: 0.0820/1.2759 | BalAcc: 0.8138 | MelSens: 0.7566 | F1: 0.8082 | LR: 0.000100 | ⏱ 70s


INFO: Saved training curves: results/training_curves_efficientnet_b0.png



✅ Resumed training complete!
   Best Balanced Acc: 0.8265


In [6]:
# ============================================================
# UTILITY: Zip and Download Checkpoints
# ============================================================
from IPython.display import FileLink

print("Zipping checkpoints...")
!zip -r -q /kaggle/working/efficientnet_checkpoints.zip /kaggle/working/checkpoints/

print("✅ Zip complete! Click the link below to download:")
display(FileLink(r'efficientnet_checkpoints.zip'))

Zipping checkpoints...
✅ Zip complete! Click the link below to download:


/kaggle/working/efficientnet_checkpoints.zip

INFO: Loading pretrained weights from Hugging Face hub (timm/convnext_tiny.in12k_ft_in1k)
INFO: HTTP Request: HEAD https://huggingface.co/timm/convnext_tiny.in12k_ft_in1k/resolve/main/model.safetensors "HTTP/1.1 302 Found"
INFO: HTTP Request: GET https://huggingface.co/api/models/timm/convnext_tiny.in12k_ft_in1k/xet-read-token/aa096f03029c7f0ec052013f64c819b34f8ad790 "HTTP/1.1 200 OK"


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

INFO: [timm/convnext_tiny.in12k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO: Model: convnext_tiny
INFO:   Feature dim: 768
INFO:   Params: 27,826,280
INFO:   Pretrained: True
/kaggle/working/src/training/trainer.py:81: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler(enabled=self.use_amp)
INFO: Backbone frozen (head-only training)
INFO: 
🚀 Training convnext_tiny...


Phase 1: Warming up classification head on cuda...


Epoch 1 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 1 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.3566
INFO:   Balanced Accuracy:  0.3492
INFO:   F1 Macro:           0.2630
INFO:   AUC-ROC Macro:      0.0000
INFO:   🎯 Mel Sensitivity: 0.4690
INFO:   🎯 Mel AUC-ROC:     0.0000
INFO:   ✅ New best model saved: convnext_tiny_best.pth


...

Ep 29/30 | Loss: 0.0467/0.6768 | BalAcc: 0.7883 | MelSens: 0.8909 | F1: 0.7784 | LR: 0.000050 | ⏱ 97s


Epoch 30 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 30 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.7737
INFO:   Balanced Accuracy:  0.8099
INFO:   F1 Macro:           0.7907
INFO:   AUC-ROC Macro:      0.0000
INFO:   🎯 Mel Sensitivity: 0.8481
INFO:   🎯 Mel AUC-ROC:     0.0000
INFO: 
✅ Training complete: 49.4 min


Ep 30/30 | Loss: 0.0432/0.6733 | BalAcc: 0.8099 | MelSens: 0.8481 | F1: 0.7907 | LR: 0.000050 | ⏱ 97s


/kaggle/working/src/training/trainer.py:151: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axes[2].legend()



✅ ConvNeXt-Tiny (Focal Loss) training complete!
   Best Balanced Acc: 0.8164


In [8]:
# ============================================================
# UTILITY: Zip and Download Checkpoints
# ============================================================
from IPython.display import FileLink

print("Zipping checkpoints...")
!zip -r -q /kaggle/working/efficientnet_checkpoints.zip /kaggle/working/checkpoints/

print("✅ Zip complete! Click the link below to download:")
display(FileLink(r'efficientnet_checkpoints.zip'))

Zipping checkpoints...
✅ Zip complete! Click the link below to download:


/kaggle/working/efficientnet_checkpoints.zip

In [9]:
# ============================================================
# UTILITY: Secure ConvNeXt Files
# ============================================================
from IPython.display import FileLink

print("Zipping checkpoints and results...")
!zip -r -q /kaggle/working/convnext_checkpoints.zip /kaggle/working/checkpoints/
!zip -r -q /kaggle/working/convnext_results.zip /kaggle/working/results/

print("✅ Zip complete! Click the links below to download to your laptop:")
display(FileLink(r'convnext_checkpoints.zip'))
display(FileLink(r'convnext_results.zip'))

Zipping checkpoints and results...
✅ Zip complete! Click the links below to download to your laptop:


/kaggle/working/convnext_checkpoints.zip

/kaggle/working/convnext_results.zip

In [10]:
# ============================================================
# CELL 6: Train Model 3 — ResNet50 (Industry Standard)
# ============================================================
import torch
from pathlib import Path
from src.models.classifier import build_model
from src.training.trainer import Trainer

MODEL_NAME = "resnet50"

# 1. Build the ResNet50 architecture
model = build_model(
    backbone_name=MODEL_NAME,
    num_classes=config.dataset.num_classes,
    dropout=0.3,
    pretrained=True, # Load ImageNet weights
    device=device
)

# 2. Setup the Trainer
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    class_weights=class_weights,
    config=config,
    device=device,
    model_name=MODEL_NAME,
    checkpoint_dir=config.paths.checkpoints_dir,
)

# Phase 1: Head-only warmup (5 epochs)
print(f"Phase 1: Warming up classification head on {device}...")
model.freeze_backbone()
trainer.num_epochs = 5
trainer.train()

# Phase 2: Full fine-tuning
print(f"\nPhase 2: Full fine-tuning on {device}...")
model.unfreeze_backbone()
trainer.num_epochs = config.training.num_epochs
trainer.early_stopping.counter = 0  
history_resnet = trainer.train()

print("\n✅ ResNet50 training complete!")
print(f"   Best Balanced Acc: {trainer.best_val_balanced_acc:.4f}")

INFO: Loading pretrained weights from Hugging Face hub (timm/resnet50.a1_in1k)
INFO: HTTP Request: HEAD https://huggingface.co/timm/resnet50.a1_in1k/resolve/main/model.safetensors "HTTP/1.1 302 Found"
INFO: HTTP Request: GET https://huggingface.co/api/models/timm/resnet50.a1_in1k/xet-read-token/767268603ca0cb0bfe326fa87277f19c419566ef "HTTP/1.1 200 OK"


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

INFO: [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO: Model: resnet50
INFO:   Feature dim: 2048
INFO:   Params: 23,524,424
INFO:   Pretrained: True
/kaggle/working/src/training/trainer.py:81: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler(enabled=self.use_amp)
INFO: Backbone frozen (head-only training)
INFO: 
🚀 Training resnet50...


Phase 1: Warming up classification head on cuda...


Epoch 1 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 1 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.2084
INFO:   Balanced Accuracy:  0.2134
INFO:   F1 Macro:           0.1378
INFO:   AUC-ROC Macro:      0.6844
INFO:   🎯 Mel Sensitivity: 0.4248
INFO:   🎯 Mel AUC-ROC:     0.6496
INFO:   ✅ New best model saved: resnet50_best.pth


...

Ep 28/30 | Loss: 0.1745/0.6357 | BalAcc: 0.7577 | MelSens: 0.7670 | F1: 0.6972 | LR: 0.000100 | ⏱ 87s


Epoch 29 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 29 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.6492
INFO:   Balanced Accuracy:  0.7652
INFO:   F1 Macro:           0.6793
INFO:   AUC-ROC Macro:      0.0000
INFO:   🎯 Mel Sensitivity: 0.7891
INFO:   🎯 Mel AUC-ROC:     0.0000
INFO:   ✅ New best model saved: resnet50_best.pth


Ep 29/30 | Loss: 0.1674/0.5542 | BalAcc: 0.7652 | MelSens: 0.7891 | F1: 0.6793 | LR: 0.000100 | ⏱ 87s


Epoch 30 [Train]:   0%|          | 0/555 [00:00<?, ?it/s]/kaggle/working/src/training/trainer.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
Epoch 30 [Val]:   0%|          | 0/119 [00:00<?, ?it/s]                                    /kaggle/working/src/training/trainer.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.use_amp):
INFO: 
INFO: METRICS [VAL]
INFO: ==================================================
INFO:   Accuracy:           0.6463
INFO:   Balanced Accuracy:  0.7679
INFO:   F1 Macro:           0.6810
INFO:   AUC-ROC Macro:      0.0000
INFO:   🎯 Mel Sensitivity: 0.8038
INFO:   🎯 Mel AUC-ROC:     0.0000
INFO:   ✅ New best model saved: resnet50_best.pth
INFO: 
✅ Training complete: 43.6 min
/kaggle/working/src/training/trainer.py:151: UserWarning: No

Ep 30/30 | Loss: 0.1502/0.6168 | BalAcc: 0.7679 | MelSens: 0.8038 | F1: 0.6810 | LR: 0.000100 | ⏱ 87s

✅ ResNet50 training complete!
   Best Balanced Acc: 0.7679


In [11]:
# ============================================================
# UTILITY: Secure All Final Models & Results
# ============================================================
from IPython.display import FileLink

print("Zipping all checkpoints and results...")
!zip -r -q /kaggle/working/all_checkpoints.zip /kaggle/working/checkpoints/
!zip -r -q /kaggle/working/all_results.zip /kaggle/working/results/

print("✅ Zip complete! Click the links below to download to your laptop:")
display(FileLink(r'all_checkpoints.zip'))
display(FileLink(r'all_results.zip'))

Zipping all checkpoints and results...
✅ Zip complete! Click the links below to download to your laptop:


/kaggle/working/all_checkpoints.zip

/kaggle/working/all_results.zip